In [1]:
import requests

In [2]:
api_url = "https://jsonplaceholder.typicode.com/todos/1"

In [3]:
response = requests.get(api_url)
response.json()

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}